In [48]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

In [106]:
with open('../quint/data/sample_texts/joe_rogan_tr.txt') as f:
    doc = f.readlines()

In [110]:
len(doc)

1

In [ ]:
doc = doc[0].replace("?", ". ")
sentences = doc.split('. ')

In [51]:
embeddings = model.encode(sentences)

In [52]:

# A =  embeddings
# A_sparse = sparse.csr_matrix(A)

# similarities = cosine_similarity(A_sparse)
# # print('pairwise dense output:\n {}\n'.format(similarities))

# #also can output sparse matrices
# similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# # print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [14]:
# A =  embeddings[0:0+50]
# A_sparse = sparse.csr_matrix(A)
# similarities = cosine_similarity(A_sparse)
# average_sim = similarities.mean(axis=1)
# best_sentence = list(average_sim).index(average_sim.max())
# similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [53]:
# average_sim = np.round(similarities.mean(axis=1),2)
# np.round(average_sim,2)

In [54]:
import pandas as pd

df = pd.DataFrame()
df['sentence'] = sentences[:-1]

df['len'] = df['sentence'].apply(lambda x: len(x))
df['cum'] = df['len'].cumsum()

In [99]:
best_sentences = []
steps = range(0, df.cum.max() + 4000, 8000)
for each in range(len(steps)-1):
    temp_df = df.loc[(df.cum > steps[each])&(df.cum < steps[each+1])]
    indexes = temp_df.index
    A =  embeddings[indexes[0]:indexes[-1]+1]
    A_sparse = sparse.csr_matrix(A)
    similarities = cosine_similarity(A_sparse)
    average_sim = list(np.round(similarities.mean(axis=1),2))
    min_score =np.min(average_sim)
    max_score = np.max(average_sim)
    best_sentence = [i for i, x in enumerate(average_sim) if (x > max_score-0.02) | (x == min_score)]
    for every in best_sentence:
        best_sentences.append(temp_df.iloc[every].sentence)
        
best_sentences = [each for each in best_sentences if len(each.split(' '))> 5]

In [100]:
df['highlight'] = df['sentence'].apply(lambda x: x in best_sentences)

In [101]:
# text='left foot right foot left foot one right. Feet in the day, feet at night.'
# l1=['foot one','feet']
# result = " ".join(colored(text,'white','on_red') if t in l1 else t for t in text.lower().split())
# print(result)

In [102]:
# summary = ''
# for each in best_sentences:
#     summary+= f'{each}. '
    
# summary

In [116]:
with open('../quint/data/results/Short.txt', 'rb') as f:
       doc = f.readlines()
doc

[b"You know people are like how do you live and things like that.  They're taking money from the Russians. \x1b\x1bAnd of course the answer is no but I do this for a living like I speak. \x1bI don't have a YouTube channel that where it's you know, I'm Joe Rogan, but I give speeches at universities and things like that. I do a lot of interviews and so we're recording now right on set up cool. Is it possible that you could do a YouTube channel.  Would that work.  If yeah, like I I mean if you introduce me so like I get followers. Yeah, we could do that. I'm all in that that could absolutely happen. Do you want to do that.  Is it something you want to do.  No, I mean this is a big question. "]

In [103]:
from termcolor import colored

text = ''
for num, each in enumerate(df['sentence']):
    if df['highlight'].iloc[num] == True:
        text+=colored(f'{each}. ','white','on_red')
    else:
        text+=f'{each}. '
    
print(text)

Okay. You're very professional. People are like, how do you live.  And things like that. They're taking money from the russians. And of course the answer is no, but I do this for a living, like I speak. I don't have a YouTube channel where I'm joe rogan, but I give speeches at universities and things like that. I do a lot of interviews. We're recording now, right. My own setup. Is it possible that you could do a YouTube channel.  Would that work.  Yeah, if you introduce me so I get followers. Yeah, we could do that, dude, I'm all in. That could absolutely happen. Do you want to do that.  Is that something you want to do.  No. I mean, this is a big question. So I came on because I had just written a book called permanent record, which is the story of my life, because that's what publishers make you do when you're writing your first book. But it's more than that, because I didn't just want to talk about me. It's actually about the changing of technology and the changing of government in 

In [75]:
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

pairwise sparse output:
   (0, 104)	-0.02589854
  (0, 103)	0.021266721
  (0, 102)	0.0074429987
  (0, 101)	0.005696579
  (0, 100)	0.0046391385
  (0, 99)	0.08170297
  (0, 98)	0.047602687
  (0, 97)	0.08170297
  (0, 96)	-0.016100569
  (0, 95)	0.04104335
  (0, 94)	0.052055288
  (0, 93)	0.040341523
  (0, 92)	0.28603432
  (0, 91)	0.12118529
  (0, 90)	0.1056238
  (0, 89)	0.12997746
  (0, 88)	0.09192037
  (0, 87)	0.0020450838
  (0, 86)	0.052878164
  (0, 85)	-0.0083633885
  (0, 84)	0.035215165
  (0, 83)	-0.001178159
  (0, 82)	0.105132595
  (0, 81)	0.17433318
  (0, 80)	0.21162403
  :	:
  (104, 24)	0.03408325
  (104, 23)	-0.07958412
  (104, 22)	-0.02097783
  (104, 21)	0.06452103
  (104, 20)	0.16395216
  (104, 19)	0.16861263
  (104, 18)	0.058462113
  (104, 17)	0.12590262
  (104, 16)	0.105187334
  (104, 15)	0.16208836
  (104, 14)	0.06427233
  (104, 13)	-0.074408196
  (104, 12)	-0.0098676905
  (104, 11)	0.07735178
  (104, 10)	0.037142176
  (104, 9)	-0.0681082
  (104, 8)	-0.01745296
  (104, 7)	0.02898